In [1]:
import requests
import pyodata

In [19]:
SERVICE_URL = 'https://data.cityofnewyork.us/OData.svc'

namespaces = {
    'test': 'erm2-nwe9'
}

config = pyodata.v2.model.Config(xml_namespaces=namespaces)

client = pyodata.Client(SERVICE_URL, requests.Session(), config=config)

In [23]:
client.entity_sets._entity_sets['erm2-nwe9'] = pyodata.v2.service.EntitySetProxy(client.entity_sets._service, )

In [25]:
try:
    client.entity_sets.__getattr__('erm2-nwe9').get_entities().count().execute()
except pyodata.exceptions.HttpError as err:
    x = err.response.request

In [17]:
str(x.url)

'https://data.cityofnewyork.us/OData.svc/311%20Service%20Requests%20from%202010%20to%20Present/$count'